In [1]:
#Se quiere importar un archivo pickle con un diccionario y transformarlo en un dataframe de pandas

import pandas as pd
import pickle
import re
#Se importa el archivo pickle
with open('scrap_info_alq.pkl', 'rb') as f:
    diccionario = pickle.load(f)


with open('scrap_info.pkl', 'rb') as f:
    diccionario2 = pickle.load(f)
#Se transforma el diccionario en un dataframe
df = pd.DataFrame(diccionario)
df2 = pd.DataFrame(diccionario2)
#Ahora se cambia las filas por las columnas
df = df.T
df2 = df2.T
df.head()
df2.head()

#Ahora unimos los dos dataframes
df_def = pd.concat([df,df2])
#Limpiamos la columna de precio quitando el simbolo de € y quitando el /mes

df_def['Precio'] = df_def['Precio'].str.replace('€','')
df_def['Precio'] = df_def['Precio'].str.replace('/mes','')

#transformamos el el precio a numerico	
df_def['Precio'] = df_def['Precio'].str.replace('.','')

#Eliminamos la fila que Precio que pone por consultar
df_def = df_def[df_def['Precio'] != 'precio a consultar']

#Transformamos la columna de precio a numerico
df_def['Precio'] = pd.to_numeric(df_def['Precio'])

#vemos el tipo de datos de cada columna
df_def.dtypes

c:\Users\pjcan\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\pjcan\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Precio                    int64
Descripcion              object
Otras_caractericas       object
Caracteristicas          object
Eficiencia_energetica    object
dtype: object

Tras llamar al dataset y limpiar la columna del precio se va intentar construir un dataframe con columnas de m^2 número de habitaciones, baños etc.

In [2]:
# Lista para almacenar los resultados
resultados = []

for i in range(0, len(df_def)):
    # Obtén la información sobre las habitaciones de la fila actual
    habitaciones_info = df_def["Caracteristicas"][i].get("habitaciones")
    
    # Verificar si habitaciones_info no es None
    if habitaciones_info:
        # Buscar el número que acompaña a la palabra "dormitorio"
        dormitorios_numero = re.search(r'(\d+) dormitorios', habitaciones_info)
        if dormitorios_numero:
            dormitorios = dormitorios_numero.group(1)
        else:
            dormitorios = "0"
    else:
        dormitorios = "0"
    
    # Agregar el resultado a la lista
    resultados.append(dormitorios)

# Agregar la lista como una nueva columna al DataFrame
df_def['Habitaciones'] = resultados

# Mostrar el DataFrame con la nueva columna
print(df_def["Habitaciones"])
   

https://www.indomio.es/anuncios/95227781/    3
https://www.indomio.es/anuncios/95146207/    4
https://www.indomio.es/anuncios/94613177/    2
https://www.indomio.es/anuncios/94500757/    2
https://www.indomio.es/anuncios/95223135/    0
                                            ..
https://www.indomio.es/anuncios/95007509/    3
https://www.indomio.es/anuncios/94958481/    2
https://www.indomio.es/anuncios/95006805/    2
https://www.indomio.es/anuncios/95313313/    3
https://www.indomio.es/anuncios/95319871/    2
Name: Habitaciones, Length: 1558, dtype: object


C:\Users\pjcan\AppData\Local\Temp\ipykernel_14648\2467882900.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  habitaciones_info = df_def["Caracteristicas"][i].get("habitaciones")


In [3]:
#Ahora se realiza lo mismo con los baños

resultados = []

for i in range(0, len(df_def)):
    # Obtén la información sobre las habitaciones de la fila actual
    habitaciones_info = df_def["Caracteristicas"][i].get("habitaciones")
    
    # Verificar si habitaciones_info no es None
    if habitaciones_info:
        # Buscar el número que acompaña a la palabra "dormitorio"
        dormitorios_numero = re.search(r'(\d+) baño[s]?', habitaciones_info)
        if dormitorios_numero:
            dormitorios = dormitorios_numero.group(1)
        else:
            dormitorios = "0"
    else:
        dormitorios = "0"
    
    # Agregar el resultado a la lista
    resultados.append(dormitorios)

# Agregar la lista como una nueva columna al DataFrame
df_def['Baños'] = resultados


C:\Users\pjcan\AppData\Local\Temp\ipykernel_14648\1420532621.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  habitaciones_info = df_def["Caracteristicas"][i].get("habitaciones")


In [4]:
import re

metros_cuadrados_lista = []
for i in range(0, len(df_def)):
    # Convertir la entrada de la columna "Descripcion" a cadena de texto
    texto = str(df_def["Descripcion"][i])
    
    # Buscar los metros cuadrados usando una expresión regular
    metros_cuadrados_match = re.search(r'(\d+(?:\.\d+)?)\s*(?:m²|m2|metros\s+cuadrados|metros)', texto)
    
    # Verificar si se encontró la información de los metros cuadrados
    if metros_cuadrados_match:
        metros_cuadrados = float(metros_cuadrados_match.group(1))  # Convertir a float en lugar de int
    else:
        metros_cuadrados = 0
    
    # Agregar el resultado a la lista
    metros_cuadrados_lista.append(metros_cuadrados)

print(metros_cuadrados_lista)

#Ahora transformamos la lista en una columna del dataframe que sea metros cuadrados
df_def["Metros cuadrados"] = metros_cuadrados_lista



[0, 120.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 99.0, 0, 130.0, 0, 161.0, 134.0, 0, 0, 175.0, 0, 170.0, 0, 161.0, 0, 0, 0, 0, 0, 130.0, 0, 0, 90.0, 0, 0, 800.0, 0, 0, 0, 90.0, 120.0, 0, 0, 0, 2.0, 0, 0, 165.0, 2.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 30.0, 200.0, 200.0, 0, 46.0, 75.0, 0, 0, 0, 0, 0, 120.0, 0, 0, 0, 0, 0, 85.0, 0, 0, 13.0, 0, 13.0, 105.0, 0, 0, 0, 0, 120.0, 0, 0, 13.0, 13.0, 180.0, 10.0, 0, 110.0, 75.0, 13.0, 0, 125.0, 70.0, 166.0, 0, 0, 0, 93.0, 0, 125.0, 125.0, 160.0, 125.0, 90.0, 0, 55.0, 0, 0, 12.0, 90.0, 0, 0, 138.0, 81.0, 0, 0, 0, 0, 75.0, 0, 0, 88.0, 0, 0, 0, 0, 15.0, 0, 0, 150.0, 93.0, 0, 0, 0, 0, 0, 0, 40.0, 100.0, 0, 0, 80.0, 0, 0, 0, 50.0, 0, 0, 0, 0, 0, 0, 79.0, 0, 135.0, 0, 50.0, 80.0, 0, 76.0, 0, 0, 95.0, 0, 150.0, 0, 0, 4.0, 0, 208.0, 0, 0, 0, 100.0, 0, 0, 0, 0, 0, 0, 18.0, 0, 0, 0, 90.0, 0, 90.0, 0, 0, 90.0, 0, 0, 0, 80.0, 0, 0, 60.0, 500.0, 98.0, 0, 70.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 63.0, 0, 0, 0, 50.0, 0, 80.0, 0, 0, 0, 50.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 70.0, 90.0, 75.0, 0

C:\Users\pjcan\AppData\Local\Temp\ipykernel_14648\1454294714.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  texto = str(df_def["Descripcion"][i])


In [5]:
#Ahora se va a visualizar el tipo de contrato que se tiene en cada piso.


#Se crea una lista para almacenar los resultados
resultados = []
for i in range(0, len(df_def)):
    contrato = df_def["Caracteristicas"][i].get("contrato")
    
    if contrato:
        contrato = contrato.lower()
    else:
        contrato = "no especificado"
    
    resultados.append(contrato)

df_def["Contrato"] = resultados




C:\Users\pjcan\AppData\Local\Temp\ipykernel_14648\3358648040.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  contrato = df_def["Caracteristicas"][i].get("contrato")


In [6]:

#Se cambia el tipo de datos de las columnas habitaciones, baños a numerico y el de contrato a stirng
df_def["Habitaciones"] = pd.to_numeric(df_def["Habitaciones"])
df_def["Baños"] = pd.to_numeric(df_def["Baños"])
df_def["Contrato"] = df_def["Contrato"].astype(str)


df_def.dtypes

Precio                     int64
Descripcion               object
Otras_caractericas        object
Caracteristicas           object
Eficiencia_energetica     object
Habitaciones               int64
Baños                      int64
Metros cuadrados         float64
Contrato                  object
dtype: object

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd
import re
import unicodedata
import spacy
import string
from spellchecker import SpellChecker   #conda install -c conda-forge pyspellchecker
# Spanish
spell = SpellChecker(language='es')  
nlp_es = spacy.load('es_core_news_md')

# Eliminamos caracteres especiales
def remove_special_characters(text):
    pat = '[{}]'.format(re.escape(string.punctuation))
    return re.sub(pat, '', text)

# Quitamos puntuación/espacios y stopwords, lematizamos si lemma=TRUE y lo ponemos en minusculas
def process_text(doc, lemma=True):
    tokens = nlp_es(doc)
    filtered_tokens = [t.lemma_.lower() if lemma else t.lower_ for t in tokens if not t.is_stop and not t.is_punct and not t.is_space]
    return ' '.join(filtered_tokens)

# Corregimos palabras mal escritas
def correct_spelling(filtered_tokens):
    corrected_tokens = []
    for token in filtered_tokens:
        if token.isalpha():
            corrected_word = spell.correction(token)
            corrected_tokens.append(corrected_word if corrected_word else token)
        else:
            corrected_tokens.append(token)
    return ''.join(corrected_tokens)

# Eliminamos acentos así como cualquier caracter que no están codificados en ASCII
def remove_accents(text):
    new_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return new_text

def preprocess_text(text):
    if text is None:
        return ''
    text = remove_special_characters(text)
    text = correct_spelling(text)
    text = process_text(text, lemma=False)
    text = remove_accents(text)
    return text

df_def['Descripcion'] = df_def['Descripcion'].apply(preprocess_text)

print(df_def)

# tfidf_vectorizer = TfidfVectorizer(max_features=1000)
# X = tfidf_vectorizer.fit_transform(df_def['Descripcion'])
# print(X)

                                           Precio  \
https://www.indomio.es/anuncios/95227781/    1155   
https://www.indomio.es/anuncios/95146207/    1540   
https://www.indomio.es/anuncios/94613177/    2500   
https://www.indomio.es/anuncios/94500757/    2500   
https://www.indomio.es/anuncios/95223135/     550   
...                                           ...   
https://www.indomio.es/anuncios/95007509/  265000   
https://www.indomio.es/anuncios/94958481/  230000   
https://www.indomio.es/anuncios/95006805/  225000   
https://www.indomio.es/anuncios/95313313/  139000   
https://www.indomio.es/anuncios/95319871/  115000   

                                                                                 Descripcion  \
https://www.indomio.es/anuncios/95227781/  alquila piso amueblado 3 habitaciones carrer d...   
https://www.indomio.es/anuncios/95146207/  piso estudiantes entrar 1ro febrero 30 junio p...   
https://www.indomio.es/anuncios/94613177/  disponible alquiler mensual anua

In [8]:
print(df_def["Descripcion"])
columns = ['Descripcion', 'Precio', 'Habitaciones', 'Baños', 'Metros cuadrados', 'Contrato']
df_filtered = df_def[columns]

df_filtered.to_pickle("pisos_filtrado.pkl")

https://www.indomio.es/anuncios/95227781/    alquila piso amueblado 3 habitaciones carrer d...
https://www.indomio.es/anuncios/95146207/    piso estudiantes entrar 1ro febrero 30 junio p...
https://www.indomio.es/anuncios/94613177/    disponible alquiler mensual anual 2 dormitorio...
https://www.indomio.es/anuncios/94500757/    disponible alquiler mensual anual 2 dormitorio...
https://www.indomio.es/anuncios/95223135/    rincon acogedor necesito espacio gustara perso...
                                                                   ...                        
https://www.indomio.es/anuncios/95007509/    atico 95 m2 valencia valencia solidinmuebles v...
https://www.indomio.es/anuncios/94958481/    venden pisos obra camino moncada residencial 1...
https://www.indomio.es/anuncios/95006805/    vende exclusiva 6 viviendas obra calle arquite...
https://www.indomio.es/anuncios/95313313/    vende piso barrio tormoseste luminoso apartame...
https://www.indomio.es/anuncios/95319871/    piso 

In [9]:
# Obtener los 5 pisos más comunes basados en la columna 'Descripcion', "Precio", "Habitaciones", "Baños", "Metros cuadrados" y "Contrato"

# Seleccionar las columnas relevantes
columns = ['Descripcion', 'Precio', 'Habitaciones', 'Baños', 'Metros cuadrados', 'Contrato']
df_filtered = df_def[columns]

#descripcion = "con terraza y vistas al mar"
min_bedrooms = 2
max_price = 1500
min_bathrooms = 1
min_sqft = 120
contrato = "alquiler"

# Filtrar el DataFrame por habitaciones, precio, baño y metros cuadrados
filtered_df = df_filtered[(df_filtered['Habitaciones'] >= min_bedrooms) &
                          (df_filtered['Precio'] <= max_price) &
                          (df_filtered['Baños'] >= min_bathrooms) &
                          (df_filtered['Metros cuadrados'] >= min_sqft) &
                          (df_filtered['Contrato'] == contrato)]
                          #(df_filtered['Descripcion'].str.contains(descripcion))]

# Crear un vectorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_filtered = tfidf_vectorizer.fit_transform(filtered_df['Descripcion'])

# Calcular la similitud de coseno entre las descripciones
similarities = cosine_similarity(X_filtered)

# Calcular la similitud total para cada piso
total_similarities = similarities.sum(axis=1)

# Obtener los índices de los 5 pisos más comunes
top_indices = total_similarities.argsort()[::-1][:5]

# Mostrar los 5 pisos más comunes
print(df_filtered.iloc[top_indices])


                                                                                 Descripcion  \
https://www.indomio.es/anuncios/94500757/  disponible alquiler mensual anual 2 dormitorio...   
https://www.indomio.es/anuncios/94613177/  disponible alquiler mensual anual 2 dormitorio...   
https://www.indomio.es/anuncios/95211299/  encantador balcon necesitan vista gustara depe...   
https://www.indomio.es/anuncios/95146207/  piso estudiantes entrar 1ro febrero 30 junio p...   
https://www.indomio.es/anuncios/95259833/  perspectiva tendencia vista elegante apartamen...   

                                           Precio  Habitaciones  Baños  \
https://www.indomio.es/anuncios/94500757/    2500             2      2   
https://www.indomio.es/anuncios/94613177/    2500             2      1   
https://www.indomio.es/anuncios/95211299/     900             0      1   
https://www.indomio.es/anuncios/95146207/    1540             4      1   
https://www.indomio.es/anuncios/95259833/     900    